In [1]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10
import re

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

dfPairs = pd.DataFrame()
dfPairs = pd.read_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_5_K_Means_cluster_3_Pvalue_02.csv')
ListOfPairsToCheck = dfPairs['Pairs'].tolist()

In [3]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks16-17-18.csv')
ListOfSnPCompanies = df['Name'].tolist()
ListOfSnPCompanies = list(set(ListOfSnPCompanies))


In [4]:
for i in range(0,len(ListOfSnPCompanies)):
    if ListOfSnPCompanies[i]=='BRK.B':
        ListOfSnPCompanies[i] = 'BRKB'
    if ListOfSnPCompanies[i]=='BF.B':
        ListOfSnPCompanies[i] = 'BFB'


In [5]:
dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""
count = 0

#Making the dat dictionary 
for c in ListOfSnPCompanies:
    dfcomp = df.loc[df['Name'] == c]
    listComp = dfcomp['close'].values
    dat[c] = listComp



In [6]:
listOfPairsFinal = []
for pair in ListOfPairsToCheck:
    pair = pair.replace('(',"")
    pair = pair.replace(')',"")
    pair = pair.replace(" ", "")
    pair = pair.split(',')
    pair[0] = re.sub('[^A-Za-z0-9]+', '', pair[0])    
    pair[1] = re.sub('[^A-Za-z0-9]+', '', pair[1])
    listOfPairsFinal.append(pair)


In [7]:
import csv

count = 0
#s_and_p = 
#dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""

for pair in listOfPairsFinal:
    print(pair)
    in_market = 0
    short_pos = 0
    equity_curve = float(100)
    ret = []
    f_ret = []
    a_ret = []
    b_ret = []
    zdiff = []
    signal = [] #shod i long or short
    lCap5 = []
    sCap5 =[]
    age = [0]   #should be -1 indexed
    aged = []
    pos = [0]   #should be -1 indexed .. signal was +1 but long cap has been reached then pos = 0 .. im not in market
    for i in range (5,212):
        a_ret.append((dat[pair[0]][i]-dat[pair[0]][i-5])/dat[pair[0]][i-5])
        b_ret.append((dat[pair[1]][i]-dat[pair[1]][i-5])/dat[pair[1]][i-5])
    for j in range (59,206):
        zdiff.append(a_ret[j+1]/np.std(a_ret[0:j+1])-b_ret[j+1]/np.std(b_ret[0:j+1]))
    for k in range (65,211):
        f_ret.append((dat[pair[0]][k+1]-dat[pair[0]][k])/dat[pair[0]][k]-(dat[pair[1]][k+1]-dat[pair[1]][k])/dat[pair[1]][k])
    for l in range (0,len(zdiff)):
        if zdiff[l] < longEntryThr:
            signal.append(1)
        elif zdiff[l] > shortEntryThr:
            signal.append(-1)
        else:
            signal.append(0)

        if pos[l] == 1 and zdiff[l] > longCap:
            lCap5.append(1)
        else:
            lCap5.append(0)
        if pos[l] == -1 and zdiff[l] < shortCap:
            sCap5.append(1)
        else:
            sCap5.append(0)

        if signal[l]!=0:
            if pos[l] == 0:
                age.append(1)
            else:
                if pos[l]+signal[l] == 0:
                    age.append(1)
                else:
                    #check here if age should be age + 1
                    age.append(1)
        else:
            if age[l] == 5:
                age.append(0)
            else:
                if pos[l] == 1 and lCap5[l] != 0 or pos[l] == -1 and sCap5[l] != 0:
                    age.append(0)
                else:
                    if signal[l] + pos[l] != 0:
                        age.append(age[l] + 1)
                    else:
                        age.append(0)

        if age[l] == 5:
            if signal[l] == 0:
                aged.append(1)
            else:
                aged.append(0)
        else:
            aged.append(0)

        if signal[l] == 0:
            if aged[l] + sCap5[l] + lCap5[l] == 0:
                pos.append(pos[l])
            else:
                if aged[l] + sCap5[l] > 0 and pos[l] == -1:
                    pos.append(0)
                elif aged[l] + lCap5[l] > 0 and pos[l] == 1:
                    pos.append(0)
                else:
                    pos.append(pos[l])
        else:
            if signal[l]!=pos[l]:
                pos.append(signal[l])
            else:
                if aged[l] + sCap5[l] + lCap5[l] == 0:
                    pos.append(signal[l])
                else:
                    if aged[l] + sCap5[l] > 0 and pos[l] == -1  and signal[l] == -1:
                        pos.append(0)
                    elif aged[l] + lCap5[l] > 0 and pos[l] == 1 and signal[l] == 1:
                        pos.append(0)
                    else:
                        pos.append(signal[l])
    count = 0
        
    countOfPosResults = 0
    countOfNegResults = 0
    countOfTotalResults = 0
    
    for m in range(1,len(pos)-1):
        if(pos[m]*f_ret[m-1]>0):
            count = count + 1
        elif (pos[m]*f_ret[m-1]<0):
            count = count -1
        ret.append(pos[m]*f_ret[m-1])
        equity_curve = equity_curve*(1+pos[m]*f_ret[m-1])
        if pos[m]!=0:
            in_market = in_market + 1
        if pos[m]==-1:
            short_pos = short_pos + 1
    details = {}
    details["Sharpe"] = np.sqrt(260)*np.mean(ret)/np.std(ret)
    details["CumRet"] = (equity_curve-100)/100
    details["In_market"] = in_market/440
    details["Short"] = short_pos/440
    done_pair.append((pair[0], pair[1], details))

    if details["Sharpe"]>0 and details["CumRet"]>0:
        pos_result = pos_result  + str(done_pair[-1]) + "\n\n"
    elif details["Sharpe"]<0 and details["CumRet"]<0:
        neg_result = neg_result  + str(done_pair[-1]) + "\n\n"
    else:
        mixed_result = mixed_result  + str(done_pair[-1]) + "\n\n"
    result = result + str(done_pair[-1]) + "\n\n"
    
file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/pos_analysis_Cluster_3_Pvalue_02.txt', 'w')
file.write(pos_result)
file.close()


['A', 'AMD']
['A', 'APA']
['A', 'EW']
['A', 'FLR']
['A', 'FLS']
['A', 'NBL']
['A', 'TRIP']
['AAL', 'MU']
['AAP', 'ALK']
['AAP', 'AMD']
['AAP', 'CMG']
['AAP', 'EXPE']
['AAP', 'NEM']
['ABBV', 'ADM']
['ABBV', 'AES']
['ABBV', 'AET']
['ABBV', 'AMD']
['ABBV', 'AMG']
['ABBV', 'AMP']
['ABBV', 'ANTM']
['ABBV', 'AON']
['ABBV', 'ATVI']
['ABBV', 'AVGO']
['ABBV', 'BEN']
['ABBV', 'CF']
['ABBV', 'CI']
['ABBV', 'CMG']
['ABBV', 'COF']
['ABBV', 'COG']
['ABBV', 'CSX']
['ABBV', 'DE']
['ABBV', 'DG']
['ABBV', 'FDX']
['ABBV', 'FFIV']
['ABBV', 'GD']
['ABBV', 'GILD']
['ABBV', 'GS']
['ABBV', 'HST']
['ABBV', 'INTU']
['ABBV', 'IT']
['ABBV', 'JWN']
['ABBV', 'LNC']
['ABBV', 'LRCX']
['ABBV', 'MCO']
['ABBV', 'MET']
['ABBV', 'MMC']
['ABBV', 'MS']
['ABBV', 'NVDA']
['ABBV', 'PCAR']
['ABBV', 'PH']
['ABBV', 'PRGO']
['ABBV', 'PRU']
['ABBV', 'PVH']
['ABBV', 'PXD']
['ABBV', 'REGN']
['ABBV', 'RL']
['ABBV', 'SCHW']
['ABBV', 'SEE']
['ABBV', 'STT']
['ABBV', 'STX']
['ABBV', 'TIF']
['ABBV', 'TWX']
['ABBV', 'VFC']
['ABBV', 'VRTX']


['ADBE', 'CB']
['ADBE', 'CBOE']
['ADBE', 'CBS']
['ADBE', 'CCL']
['ADBE', 'CERN']
['ADBE', 'CF']
['ADBE', 'CHRW']
['ADBE', 'CINF']
['ADBE', 'CL']
['ADBE', 'CMA']
['ADBE', 'CMCSA']
['ADBE', 'CMI']
['ADBE', 'CNP']
['ADBE', 'COF']
['ADBE', 'COG']
['ADBE', 'COP']
['ADBE', 'CPB']
['ADBE', 'CRM']
['ADBE', 'CSX']
['ADBE', 'CTL']
['ADBE', 'CTXS']
['ADBE', 'CVS']
['ADBE', 'CVX']
['ADBE', 'CXO']
['ADBE', 'DE']
['ADBE', 'DFS']
['ADBE', 'DG']
['ADBE', 'DGX']
['ADBE', 'DHI']
['ADBE', 'DIS']
['ADBE', 'DISCK']
['ADBE', 'DISH']
['ADBE', 'DOV']
['ADBE', 'DRI']
['ADBE', 'DTE']
['ADBE', 'DUK']
['ADBE', 'DVA']
['ADBE', 'DVN']
['ADBE', 'EBAY']
['ADBE', 'ECL']
['ADBE', 'ED']
['ADBE', 'EFX']
['ADBE', 'EL']
['ADBE', 'EMN']
['ADBE', 'EMR']
['ADBE', 'EOG']
['ADBE', 'EQT']
['ADBE', 'ESRX']
['ADBE', 'ESS']
['ADBE', 'ETN']
['ADBE', 'ETR']
['ADBE', 'EXC']
['ADBE', 'F']
['ADBE', 'FAST']
['ADBE', 'FBHS']
['ADBE', 'FE']
['ADBE', 'FFIV']
['ADBE', 'FIS']
['ADBE', 'FISV']
['ADBE', 'FITB']
['ADBE', 'FLIR']
['ADBE', 'FLR']


['AEE', 'DOV']
['AEE', 'EL']
['AEE', 'FTI']
['AEE', 'GPC']
['AEE', 'GS']
['AEE', 'HCA']
['AEE', 'HOLX']
['AEE', 'HSY']
['AEE', 'IBM']
['AEE', 'IDXX']
['AEE', 'LKQ']
['AEE', 'LLL']
['AEE', 'LYB']
['AEE', 'MSI']
['AEE', 'NEM']
['AEE', 'NFX']
['AEE', 'NSC']
['AEE', 'PKG']
['AEE', 'PLD']
['AEE', 'PWR']
['AEE', 'RCL']
['AEE', 'RTN']
['AEE', 'SLB']
['AEE', 'SPG']
['AEE', 'SYY']
['AEE', 'TRIP']
['AEE', 'TXN']
['AEE', 'UAL']
['AEE', 'UHS']
['AEE', 'V']
['AEE', 'VNO']
['AEE', 'WEC']
['AEE', 'WU']
['AEE', 'YUM']
['AEE', 'ZION']
['AEP', 'AES']
['AEP', 'AFL']
['AEP', 'AIG']
['AEP', 'AKAM']
['AEP', 'ALK']
['AEP', 'ALL']
['AEP', 'ALXN']
['AEP', 'AMG']
['AEP', 'AMT']
['AEP', 'ANSS']
['AEP', 'ATVI']
['AEP', 'AVGO']
['AEP', 'AXP']
['AEP', 'BDX']
['AEP', 'BSX']
['AEP', 'BWA']
['AEP', 'BXP']
['AEP', 'C']
['AEP', 'CB']
['AEP', 'CBOE']
['AEP', 'CBS']
['AEP', 'CDNS']
['AEP', 'CI']
['AEP', 'CL']
['AEP', 'CMA']
['AEP', 'COF']
['AEP', 'COO']
['AEP', 'CRM']
['AEP', 'CTXS']
['AEP', 'CXO']
['AEP', 'DAL']
['AEP', 

['AGN', 'MRK']
['AGN', 'MS']
['AGN', 'MTD']
['AGN', 'NEM']
['AGN', 'NI']
['AGN', 'NOC']
['AGN', 'NSC']
['AGN', 'OMC']
['AGN', 'ORLY']
['AGN', 'PCAR']
['AGN', 'PH']
['AGN', 'PKG']
['AGN', 'PKI']
['AGN', 'PNC']
['AGN', 'PNR']
['AGN', 'PPG']
['AGN', 'PRGO']
['AGN', 'RCL']
['AGN', 'REGN']
['AGN', 'RHI']
['AGN', 'RHT']
['AGN', 'RJF']
['AGN', 'ROK']
['AGN', 'RTN']
['AGN', 'SCHW']
['AGN', 'SEE']
['AGN', 'SNPS']
['AGN', 'SPGI']
['AGN', 'STI']
['AGN', 'STX']
['AGN', 'STZ']
['AGN', 'SYK']
['AGN', 'TDG']
['AGN', 'TEL']
['AGN', 'TMO']
['AGN', 'TSN']
['AGN', 'TWX']
['AGN', 'TXN']
['AGN', 'TXT']
['AGN', 'UAL']
['AGN', 'UNP']
['AGN', 'USB']
['AGN', 'UTX']
['AGN', 'VAR']
['AGN', 'VFC']
['AGN', 'VIAB']
['AGN', 'VLO']
['AGN', 'VMC']
['AGN', 'VNO']
['AGN', 'VRSK']
['AGN', 'WBA']
['AGN', 'WDC']
['AGN', 'WFC']
['AGN', 'WHR']
['AGN', 'WMB']
['AGN', 'WYNN']
['AGN', 'XEC']
['AGN', 'XRX']
['AGN', 'XYL']
['AGN', 'YUM']
['AGN', 'ZBH']
['AGN', 'ZTS']
['AIG', 'AJG']
['AIG', 'ALL']
['AIG', 'AME']
['AIG', 'AMT']
['A

['ALB', 'CMG']
['ALB', 'CMS']
['ALB', 'CNC']
['ALB', 'CNP']
['ALB', 'COF']
['ALB', 'COL']
['ALB', 'COO']
['ALB', 'COST']
['ALB', 'CPB']
['ALB', 'CRM']
['ALB', 'CSX']
['ALB', 'CTAS']
['ALB', 'CTSH']
['ALB', 'CTXS']
['ALB', 'CVS']
['ALB', 'CVX']
['ALB', 'D']
['ALB', 'DAL']
['ALB', 'DE']
['ALB', 'DFS']
['ALB', 'DG']
['ALB', 'DGX']
['ALB', 'DHI']
['ALB', 'DIS']
['ALB', 'DISH']
['ALB', 'DLTR']
['ALB', 'DOV']
['ALB', 'DPS']
['ALB', 'DRI']
['ALB', 'DTE']
['ALB', 'DUK']
['ALB', 'DVA']
['ALB', 'EA']
['ALB', 'ECL']
['ALB', 'ED']
['ALB', 'EFX']
['ALB', 'EIX']
['ALB', 'EL']
['ALB', 'EMR']
['ALB', 'EQR']
['ALB', 'ESRX']
['ALB', 'ESS']
['ALB', 'ETFC']
['ALB', 'ETN']
['ALB', 'ETR']
['ALB', 'EW']
['ALB', 'EXC']
['ALB', 'EXPD']
['ALB', 'EXPE']
['ALB', 'EXR']
['ALB', 'FAST']
['ALB', 'FBHS']
['ALB', 'FDX']
['ALB', 'FE']
['ALB', 'FFIV']
['ALB', 'FIS']
['ALB', 'FISV']
['ALB', 'FITB']
['ALB', 'FL']
['ALB', 'FLIR']
['ALB', 'FLR']
['ALB', 'FRT']
['ALB', 'GD']
['ALB', 'GE']
['ALB', 'GGP']
['ALB', 'GILD']
['ALB

...

['NKE', 'PVH']
['NKE', 'PWR']
['NKE', 'PX']
['NKE', 'PXD']
['NKE', 'QCOM']
['NKE', 'REGN']
['NKE', 'RF']
['NKE', 'SLB']
['NKE', 'STX']
['NKE', 'SYMC']
['NKE', 'T']
['NKE', 'TJX']
['NKE', 'UNP']
['NKE', 'UPS']
['NKE', 'UTX']
['NKE', 'V']
['NKE', 'VIAB']
['NKE', 'VTR']
['NKE', 'VZ']
['NKE', 'WDC']
['NKE', 'WMT']
['NKE', 'XOM']
['NKE', 'XRAY']
['NKE', 'YUM']
['NLSN', 'PSX']
['NLSN', 'STZ']
['NLSN', 'SYMC']
['NLSN', 'TGT']
['NLSN', 'ULTA']
['NLSN', 'VLO']
['NLSN', 'VRTX']
['NLSN', 'XYL']
['NOC', 'OMC']
['NOC', 'OXY']
['NOC', 'PKG']
['NOC', 'PM']
['NOC', 'PPG']
['NOC', 'PWR']
['NOC', 'RCL']
['NOC', 'REGN']
['NOC', 'ROST']
['NOC', 'STZ']
['NOC', 'TJX']
['NOC', 'TSN']
['NOC', 'TXN']
['NOC', 'UAL']
['NOC', 'UTX']
['NOC', 'VLO']
['NOC', 'WHR']
['NOC', 'WMB']
['NOC', 'ZBH']
['NSC', 'XRX']
['NTAP', 'VLO']
['NTRS', 'NWL']
['NTRS', 'OKE']
['NTRS', 'PFE']
['NTRS', 'PGR']
['NTRS', 'PHM']
['NTRS', 'PLD']
['NTRS', 'PM']
['NTRS', 'PNW']
['NTRS', 'PPL']
['NTRS', 'PSA']
['NTRS', 'PX']
['NTRS', 'QCOM']
['N

['PFE', 'SHW']
['PFE', 'SLB']
['PFE', 'SNA']
['PFE', 'SNPS']
['PFE', 'SO']
['PFE', 'SRCL']
['PFE', 'STI']
['PFE', 'STT']
['PFE', 'STZ']
['PFE', 'SYK']
['PFE', 'SYMC']
['PFE', 'T']
['PFE', 'TAP']
['PFE', 'TDG']
['PFE', 'TGT']
['PFE', 'TIF']
['PFE', 'TJX']
['PFE', 'TPR']
['PFE', 'TROW']
['PFE', 'TSCO']
['PFE', 'TXT']
['PFE', 'UNH']
['PFE', 'UNM']
['PFE', 'V']
['PFE', 'VAR']
['PFE', 'VFC']
['PFE', 'VMC']
['PFE', 'VRSK']
['PFE', 'WAT']
['PFE', 'WEC']
['PFE', 'WMT']
['PFE', 'XEL']
['PFE', 'XOM']
['PFE', 'YUM']
['PFE', 'ZION']
['PFG', 'PWR']
['PFG', 'QCOM']
['PFG', 'TWX']
['PFG', 'UHS']
['PG', 'RL']
['PG', 'VMC']
['PG', 'WY']
['PG', 'XRX']
['PGR', 'PHM']
['PGR', 'PKI']
['PGR', 'PNR']
['PGR', 'PSX']
['PGR', 'PWR']
['PGR', 'PX']
['PGR', 'QCOM']
['PGR', 'REGN']
['PGR', 'RF']
['PGR', 'RHT']
['PGR', 'RJF']
['PGR', 'SLB']
['PGR', 'STX']
['PGR', 'SWKS']
['PGR', 'TDG']
['PGR', 'TEL']
['PGR', 'TMO']
['PGR', 'TRIP']
['PGR', 'TSS']
['PGR', 'TXT']
['PGR', 'UAL']
['PGR', 'UNP']
['PGR', 'UTX']
['PGR', 'VA

['RCL', 'TJX']
['RCL', 'TMK']
['RCL', 'TMO']
['RCL', 'TPR']
['RCL', 'TRIP']
['RCL', 'TROW']
['RCL', 'TRV']
['RCL', 'TSCO']
['RCL', 'TSN']
['RCL', 'TWX']
['RCL', 'TXN']
['RCL', 'TXT']
['RCL', 'UAL']
['RCL', 'UHS']
['RCL', 'UNH']
['RCL', 'UNM']
['RCL', 'UNP']
['RCL', 'UPS']
['RCL', 'URI']
['RCL', 'USB']
['RCL', 'UTX']
['RCL', 'V']
['RCL', 'VAR']
['RCL', 'VFC']
['RCL', 'VIAB']
['RCL', 'VNO']
['RCL', 'VRSK']
['RCL', 'VRSN']
['RCL', 'VRTX']
['RCL', 'VTR']
['RCL', 'VZ']
['RCL', 'WAT']
['RCL', 'WBA']
['RCL', 'WDC']
['RCL', 'WEC']
['RCL', 'WFC']
['RCL', 'WHR']
['RCL', 'WM']
['RCL', 'WMB']
['RCL', 'WMT']
['RCL', 'WU']
['RCL', 'WY']
['RCL', 'WYNN']
['RCL', 'XEC']
['RCL', 'XEL']
['RCL', 'XL']
['RCL', 'XLNX']
['RCL', 'XOM']
['RCL', 'XRAY']
['RCL', 'XRX']
['RCL', 'XYL']
['RCL', 'YUM']
['RCL', 'ZBH']
['RCL', 'ZION']
['RCL', 'ZTS']
['RE', 'REGN']
['RE', 'RL']
['RE', 'SCG']
['RE', 'SCHW']
['RE', 'SNA']
['RE', 'SO']
['RE', 'SRCL']
['RE', 'SRE']
['RE', 'STT']
['RE', 'SYY']
['RE', 'TGT']
['RE', 'TMK']
['

['SNPS', 'VZ']
['SNPS', 'WEC']
['SNPS', 'WMT']
['SNPS', 'XEL']
['SNPS', 'XOM']
['SNPS', 'YUM']
['SNPS', 'ZION']
['SO', 'SPG']
['SO', 'SPGI']
['SO', 'TMK']
['SO', 'TSS']
['SO', 'UAL']
['SO', 'UHS']
['SO', 'UNM']
['SO', 'VRSK']
['SO', 'WU']
['SO', 'XEL']
['SO', 'YUM']
['SO', 'ZION']
['SPG', 'STT']
['SPG', 'SYY']
['SPG', 'TDG']
['SPG', 'TPR']
['SPG', 'TRIP']
['SPG', 'TSN']
['SPG', 'UAL']
['SPG', 'UHS']
['SPG', 'UNM']
['SPG', 'VMC']
['SPG', 'WEC']
['SPG', 'WY']
['SPG', 'XEL']
['SPG', 'YUM']
['SPG', 'ZION']
['SPG', 'ZTS']
['SRCL', 'STT']
['SRCL', 'TAP']
['SRCL', 'TGT']
['SRCL', 'TRIP']
['SRCL', 'TSN']
['SRCL', 'TWX']
['SRCL', 'UAL']
['SRCL', 'UNH']
['SRCL', 'VFC']
['SRCL', 'VLO']
['SRCL', 'VRTX']
['SRCL', 'WHR']
['SRCL', 'XOM']
['SRCL', 'XYL']
['SRE', 'STI']
['SRE', 'STT']
['SRE', 'STX']
['SRE', 'SYK']
['SRE', 'SYMC']
['SRE', 'TIF']
['SRE', 'TRIP']
['SRE', 'TROW']
['SRE', 'TWX']
['SRE', 'TXN']
['SRE', 'TXT']
['SRE', 'UNH']
['SRE', 'UPS']
['SRE', 'USB']
['SRE', 'VAR']
['SRE', 'VFC']
['SRE', 

['VLO', 'WDC']
['VLO', 'WEC']
['VLO', 'WFC']
['VLO', 'WM']
['VLO', 'WMT']
['VLO', 'WU']
['VLO', 'WY']
['VLO', 'WYNN']
['VLO', 'XEL']
['VLO', 'XL']
['VLO', 'XLNX']
['VLO', 'XOM']
['VLO', 'XRX']
['VLO', 'XYL']
['VLO', 'ZBH']
['VLO', 'ZION']
['VMC', 'VNO']
['VMC', 'VRSK']
['VMC', 'VZ']
['VMC', 'WEC']
['VMC', 'WM']
['VMC', 'WMT']
['VMC', 'WY']
['VMC', 'WYNN']
['VMC', 'XEL']
['VMC', 'XL']
['VMC', 'XOM']
['VMC', 'XRX']
['VMC', 'ZBH']
['VNO', 'XRX']
['VNO', 'ZBH']
['VRSK', 'VRTX']
['VRSK', 'VTR']
['VRSK', 'VZ']
['VRSK', 'WAT']
['VRSK', 'WDC']
['VRSK', 'WEC']
['VRSK', 'WFC']
['VRSK', 'WM']
['VRSK', 'WMB']
['VRSK', 'WMT']
['VRSK', 'WU']
['VRSK', 'WY']
['VRSK', 'XEL']
['VRSK', 'XL']
['VRSK', 'XLNX']
['VRSK', 'XOM']
['VRSK', 'XRAY']
['VRSK', 'XRX']
['VRSK', 'XYL']
['VRSK', 'YUM']
['VRSK', 'ZBH']
['VRSK', 'ZION']
['VRSK', 'ZTS']
['VRSN', 'WMB']
['VRTX', 'VTR']
['VRTX', 'VZ']
['VRTX', 'WAT']
['VRTX', 'WBA']
['VRTX', 'WDC']
['VRTX', 'WEC']
['VRTX', 'WFC']
['VRTX', 'WHR']
['VRTX', 'WM']
['VRTX', 'WMB

In [8]:

listOfPosResult = pos_result.split('\n\n')
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

9397
20275
Percentage of profiting pairs in Cluster: 46.347718865598026%


In [9]:

listOfPosResult = pos_result.split('\n\n')
list11 =[]
list22 = []
listRet = []
for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    
    list11.append(list1[0])
    list22.append(list1[1])
    listRet.append(float(cumR))
    
maxVal = max(listRet)

for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    cumR = float(cumR)
    
    if(cumR==maxVal):
        print(list1[0],list1[1],round(maxVal,2))
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

NWL SCG 2.51
9397
20275
Percentage of profiting pairs in Cluster: 46.347718865598026%
